In [ ]:
import torch
from math import sqrt
import torch.nn.functional as F
from torch import nn
from transformers import AutoConfig
from transformers import AutoTokenizer

# Transformer

Attention is All You Need

Google Brain

2017

![model](https://drive.google.com/uc?id=1VcJ2iqMMvNA1HtsXQukio9V0i6nEvWMK)

## 1. Sub-Layers Implementation

### 1.1 Positional Encoding

In [3]:
class PositionalEncoding(nn.Module): 
    def __init__(self, config):
        super().__init__()
        self.token_embeddings = nn.Embedding(config.vocab_size,
                                             config.hidden_size)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings,
                                                config.hidden_size)
        self.layer_norm = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout()

    def forward(self, input_ids):
        # Create position IDs for input sequence
        seq_length = input_ids.size(1)
        position_ids = torch.arange(seq_length, dtype=torch.long).unsqueeze(0)
        # Create token and position embeddings
        token_embeddings = self.token_embeddings(input_ids) 
        position_embeddings = self.position_embeddings(position_ids)
        # Combine token and position embeddings
        embeddings = token_embeddings + position_embeddings
        embeddings = self.layer_norm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings

### 1.2 Multi-Head Attention

![multi_head_attention](https://drive.google.com/uc?id=17xC_ptWBzoRjZch1z1q7orHZxaKZYH-t)

In [4]:
class AttentionHead(nn.Module):
    def __init__(self, embed_dim, head_dim):
        super().__init__()
        self.q = nn.Linear(embed_dim, head_dim)
        self.k = nn.Linear(embed_dim, head_dim)
        self.v = nn.Linear(embed_dim, head_dim)

    def forward(self, hidden_state):
        attn_outputs = scaled_dot_product_attention(
            self.q(hidden_state), self.k(hidden_state), self.v(hidden_state), mask)
        return attn_outputs

In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, config, embed_dim, head_dim):
        super().__init__()
        embed_dim = config.hidden_size
        num_heads = config.num_attention_heads
        head_dim = embed_dim // num_heads
        self.heads = nn.ModuleList(
            [MultiHeadAttention(embed_dim, head_dim) for _ in range(num_heads)]
        )
        self.output_linear = nn.Linear(embed_dim, embed_dim)

    def forward(self, hidden_state):
        x = torch.cat([h(hidden_state) for h in self.heads], dim=-1)
        x = self.output_linear(x)
        return x


### 1.3 Scaled Dot-Product Attention

![scaled_dot_proudct_attention](https://drive.google.com/uc?id=1VLoOdj16tctQ4YLF8vfHl5Quoc-mvT3q)

In [ ]:
def scaled_dot_product_attention(query, key, value, mask=None):
    dim_k = query.size(-1)
    scores = torch.bmm(query, key.transpose(1,2)) / sqrt(dim_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, float("-inf"))
    weights = F.softmax(scores, dim=-1)
    return weights.bmm(value)

### 1.4 Position-Wise Fully Connected Feed-Forward Network

In [ ]:
class FeedForward(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.linear_1 = nn.Linear(config.hidden_size, config.intermediate_size)
        self.linear_2 = nn.Linear(config.intermediate_size, config.hidden_size)
        self.gelu = nn.GELU()
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.gelu(x)
        x = self.linear_2(x)
        x = self.dropout(x)
        return x

## 2. Encoder Implementation

![enc_dec](https://drive.google.com/uc?id=1VUvTRuCWMCsVYbVftkIcmz63VuCc0VgL)

### 2.1 Encoder Layer

In [ ]:
class TransformerEncoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        # [1, 5, 768]
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size)
        # [1, 32, 768]
        self.layer_norm_2 = nn.LayerNorm(config.hidden_size)
        self.attention = MultiHeadAttention(config)
        self.feed_forward = FeedForward(config)

    def forward(self, x):
        # apply layer norma & copy input into a q, k, v
        hidden_state = self.layer_norm_1(x)
        # apply attention with a skip connection
        x = x + self.attention(hidden_state)
        # apply feed-forward with a skip connection
        x = x + self.feed_forward(self.layer_norm_2(x))
        return x

### 2.2 Encoder

In [ ]:
class TransformerEncoder(nn.Module): 
    def __init__(self, config):
        super().__init__()
        self.embeddings = PositionalEncoding(config)
        self.layers = nn.ModuleList([TransformerEncoderLayer(config)
                                    for _ in range(config.num_hidden_layers)])

    def forward(self, x):
        x = self.embeddings(x) 
        for layer in self.layers:
            x = layer(x) 
        return x

## 3. Decoder Implementation

![enc_dec](https://drive.google.com/uc?id=1_lfa-u_O6JaAyYRR7Dtue6db3KV-kDFI)

### 3.1 Decoder Layer

In [ ]:
class TransformerDecoderLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.layer_norm_1 = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.attention = MultiHeadAttention(config)

        self.layer_norm_2 = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.enc_dec_attention = MultiHeadAttention(config)

        self.layer_norm_3 = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.feed_forward = FeedForward(config)

    def forward(self, x_dec, x_enc, target_mask, src_mask):
        # apply layer norm & copy input into a q, k, v
        hidden_state_attention = self.layer_norm_1(x_dec)
        # apply masked attention with a skip connection 
        x = x_dec + self.attention(hidden_state_attention, target_mask)

        if x_enc is not None:
            # apply layer norm & copy input into a q, k, v
            hidden_state_enc_dec = self.layer_norm_2(x)
            # apply masked attention with a skip connection
            x = x + self.enc_dec_attention(hidden_state_enc_dec, src_mask)

        # apply feed-forward with a skip connection
        x = self.feed_forward(self.layer_norm_3(x))

        return x


### 3.3 Decoder

In [ ]:
class TransformerDecoder(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.embeddings = PositionalEncoding(config)
        self.layers = nn.ModuleList([TransformerDecoderLayer(config)
                                    for _ in range(config.num_hidden_layers)])
        
        self.linear = nn.Linear()
        
    def forward(self, target, source, target_mask, src_mask):
        target = self.embeddings(target)

        for layer in self.layers:
            x = layer(target, source, target_mask, src_mask)

        out = nn.Linear(config.hidden_size, config.intermediate_size)
        return out

## 4. References

- [Attention is All You Need, 2017 - Google](https://arxiv.org/abs/1706.03762)
- [The Illustrated Transformer - Jay Alammar](http://jalammar.github.io/illustrated-transformer/)
- [Data & Optimization Code Reference - Bentrevett](https://github.com/bentrevett/pytorch-seq2seq/)

## Contenido
cubierto hasta ahora

1. Historia AI
2. Intro AI
3. Intro Transformers
4. Text clasification
5. Transformer Arquitecture

## Transformers
Opciones para cubrir (escoger 4)

* Multilingual NER
* Text Generation
* Summarization
* Q&A
* Transformers in prod
* Training transformers from scratch

## Computer Vision

## Reinforcemente Learning